## Data Overview and Preprocessing

Before proceeding to the stage of data analysis and building a supply and demand model, it is necessary to do data processing.

Data Preprocessing includes the following actions:
1. Checking the absence of rows with metrics in datasets
2. Filling in the blanks
3. Replacement null values with zero (0)

In [1]:
import pandas as pd
import tabulate

import datetime
from datetime import timedelta

from IPython.display import Markdown, display
from typing import List

In [2]:
#!pip install watermark
%load_ext watermark

In [3]:
%watermark -v -m -p pandas,tabulate,datetime,typing -g

CPython 3.6.3
IPython 7.2.0

pandas 1.1.5
tabulate 0.8.9
datetime unknown
typing 3.6.6

compiler   : GCC 4.9.2
system     : Linux
release    : 4.19.0-12-amd64
machine    : x86_64
processor  : 
CPU cores  : 56
interpreter: 64bit
Git hash   :


## Helper functions for processing the data and basic transforms

In [4]:
def df_overview(csv_path: str, date_columns: List[str]) -> pd.DataFrame():
    '''
    Function returns dataset from csv file as pd.DataFrame() and prints dataset overview 
    
    input:
    csv_path: str – path csv file with dataset
    date_columns: List[str] – date columns to parse into datetime object
    
    output: pd.DataFrame()
    '''
    df = pd.read_csv(csv_path, parse_dates=date_columns, infer_datetime_format=True)
    print(f'{csv_path}\nDataset shape: {df.shape}')
    print('\nFirst 10 rows:')
    print(tabulate.tabulate(df[:10].values, df.columns, tablefmt="pipe"))
    print(f'\nCheck for Null values:\n{df.isna().sum()}')
    return df

def find_missing_dates(df: pd.DataFrame(), date_column: str) -> List[datetime.datetime]:
    '''
    Function returns list of missing dates in specified dataframe
    
    input:
    df: pd.DataFrame() – dataframe to search for missing dates
    date_column: str – date column 
    
    output: List[datetime.datetime]
    '''
    df = df[date_column].groupby([df[date_column].dt.date]).count().to_frame()
    df.columns = ['Rows_Count']
    df = df.reset_index()
    missing_dates = list(df[date_column][df['Rows_Count'] < 24])
    print(missing_dates)
    return missing_dates
    
def create_missing_rows(df: pd.DataFrame(), 
                        date_column: str, 
                        missing_dates: List[datetime.datetime]) -> pd.DataFrame():
    '''
    Function creates dataframe of missing rows in specified dataframe
    
    input:
    df: pd.DataFrame() – dataframe with missing rows
    date_column: str – date column 
    missing_dates: List[datetime.datetime] – missing dates to fill the rows 
    
    output: updated dataframe pd.DataFrame()
    '''
    df_missing_rows = pd.DataFrame(columns= df.columns)
    hour_list = range(24)
    for date in missing_dates:
        df_hours = list(df[date_column][df[date_column].dt.date == date].dt.hour)
        for hour in hour_list:
            if hour not in df_hours:
                date = datetime.datetime.combine(date, datetime.time(hour=hour))
                df_missing_rows=df_missing_rows.append({date_column: date}, ignore_index=True)
                df_missing_rows.fillna(value=0, inplace=True)    
    print(tabulate.tabulate(df_missing_rows.values, df_missing_rows.columns, tablefmt="pipe"))
    return df_missing_rows
    
def insert_missing_row(df: pd.DataFrame(), df_missing_rows: pd.DataFrame()) -> pd.DataFrame():
    '''
    Function inserts missimg rows to specified dataframe
    
    input:
    df: pd.DataFrame() – dataframe with missing rows
    df_missing_rows: pd.DataFrame() – dataframe of missing rows
    
    output: updated dataframe pd.DataFrame()
    '''
    df = pd.concat([df, df_missing_rows], ignore_index=True)
    print(f'New dataset shape: {df.shape}')
    print(f'\nInserted row:')
    print(tabulate.tabulate(df[-len(df_missing_rows):].values, df.columns, tablefmt="pipe"))
    return df

def replace_null_with_zero(df: pd.DataFrame(), date_columns: List[str]) -> pd.DataFrame():
    '''
    Function replaces Null values with zero (0)
    
    input:
    df: pd.DataFrame() – dataframe with Null values
    date_columns: List[str] – date columns to exclude
    
    output: updated dataframe pd.DataFrame()
    '''
    for column in df.columns:
        if column not in date_columns:
            if df[column].isna().sum() > 0:
                df[column].fillna(value=0, inplace=True)
    print(f'Final check for Nulls:\n{df.isna().sum() > 0}')
    return df

### Supply Dataset

In [5]:
supply_data = df_overview(csv_path='data/Hourly_DriverActivity_1.csv', date_columns=['Date'])

data/Hourly_DriverActivity_1.csv
Dataset shape: (840, 9)

First 10 rows:
| Date                |   Active drivers |   Online (h) |   Has booking (h) |   Waiting for booking (h) |   Busy (h) |   Hours per active driver |   Rides per online hour |   Finished Rides |
|:--------------------|-----------------:|-------------:|------------------:|--------------------------:|-----------:|--------------------------:|------------------------:|-----------------:|
| 2016-12-18 23:00:00 |               52 |           18 |                 6 |                        11 |         11 |                       0.3 |                    0.67 |               12 |
| 2016-12-18 22:00:00 |               59 |           20 |                11 |                         9 |         12 |                       0.3 |                    1.4  |               28 |
| 2016-12-18 21:00:00 |               72 |           25 |                 7 |                        18 |         15 |                       0.3 |             

In [6]:
missing_dates = find_missing_dates(df=supply_data, date_column='Date')

[]


In [7]:
supply_data_new = replace_null_with_zero(df=supply_data, date_columns=['Date'])

Final check for Nulls:
Date                       False
Active drivers             False
Online (h)                 False
Has booking (h)            False
Waiting for booking (h)    False
Busy (h)                   False
Hours per active driver    False
Rides per online hour      False
Finished Rides             False
dtype: bool


### Demand Dataset

In [8]:
demand_data = df_overview(csv_path='data/Hourly_OverviewSearch_1.csv', date_columns=['Date'])

data/Hourly_OverviewSearch_1.csv
Dataset shape: (839, 4)

First 10 rows:
| Date                |   People saw 0 cars (unique) |   People saw +1 cars (unique) |   Coverage Ratio (unique) |
|:--------------------|-----------------------------:|------------------------------:|--------------------------:|
| 2016-12-18 23:00:00 |                            9 |                            32 |                        78 |
| 2016-12-18 22:00:00 |                           29 |                            64 |                        69 |
| 2016-12-18 21:00:00 |                            5 |                            39 |                        89 |
| 2016-12-18 20:00:00 |                           13 |                            48 |                        79 |
| 2016-12-18 19:00:00 |                           12 |                            77 |                        87 |
| 2016-12-18 18:00:00 |                           13 |                            62 |                        83 |
| 2016-

In [9]:
missing_dates = find_missing_dates(df=demand_data, date_column='Date')

[datetime.date(2016, 12, 7)]


In [10]:
df_missing_rows = create_missing_rows(df=demand_data, date_column='Date', missing_dates=missing_dates)

| Date                |   People saw 0 cars (unique) |   People saw +1 cars (unique) |   Coverage Ratio (unique) |
|:--------------------|-----------------------------:|------------------------------:|--------------------------:|
| 2016-12-07 05:00:00 |                            0 |                             0 |                         0 |


In [11]:
demand_data_new = insert_missing_row(df=demand_data, df_missing_rows=df_missing_rows)

New dataset shape: (840, 4)

Inserted row:
| Date                |   People saw 0 cars (unique) |   People saw +1 cars (unique) |   Coverage Ratio (unique) |
|:--------------------|-----------------------------:|------------------------------:|--------------------------:|
| 2016-12-07 05:00:00 |                            0 |                             0 |                         0 |


In [12]:
find_missing_dates(df=demand_data_new, date_column='Date')

[]


[]

### Save new datasets to csv for further analysis

In [13]:
supply_data_new.to_csv('data/Hourly_DriverActivity_1_Processed.csv', index=False)
demand_data_new.to_csv('data/Hourly_OverviewSearch_1_Processed.csv', index=False)

## Results
At this stage of data processing, the following has been done:
1. Replaced 45 empty  values  with zero (0 )in the Supply dataset in the column 'Finished Rides'
2. Inserted a missing line in the Demand dataset for 2016-12-07 05:00:00